## 复制文件

In [5]:
import glob
import shutil
print("Before:", glob.glob('example*'))
shutil.copyfile('example.txt', 'example.txt.copy')
print("After:", glob.glob('example*'))

Before: ['example_link', 'example.txt', 'example_dir']
After: ['example_link', 'example.txt', 'example_dir', 'example.txt.copy']


In [10]:
import io
import sys
import os

class VerboseStringIO(io.StringIO):
    def read(self, n=-1):
        next = super().read(n)
        print(f'read({n}) got {len(next)} bytes')
        return next
lorem_ipsum = '''Lorem ipsum dolor sit amet, consectetuer
adipiscing elit.  Vestibulum aliquam mollis dolor. Donec
vulputate nunc ut diam. Ut rutrum mi vel sem. Vestibulum
ante ipsum.'''

print('Default:')
input = VerboseStringIO(lorem_ipsum)
output = io.StringIO()
shutil.copyfileobj(input, output)
print('All at once:')
input = VerboseStringIO(lorem_ipsum)
output = io.StringIO()
shutil.copyfileobj(input, output, -1)
print('Block 256:')
input = VerboseStringIO(lorem_ipsum)
output = io.StringIO()
shutil.copyfileobj(input, output, 128)

Default:
read(16384) got 166 bytes
read(16384) got 0 bytes
All at once:
read(-1) got 166 bytes
read(-1) got 0 bytes
Block 256:
read(128) got 128 bytes
read(128) got 38 bytes
read(128) got 0 bytes


copyfile() 不管文件的类型如何，都会打开进行读取，一些特殊的文件（例如 Unix 设备）是不能使用它进行复制的。copyfile()  使用低级的方法 copyfileobj() 实现。传入 copyfile() 的参数是文件名称，而传入 copyfileobj() 是打开的文件描述符。可选的第三个参数用来设置读取块的大小。 -1表示一次性全部读取。

In [12]:
print("Before:", glob.glob('test_files/*'))
shutil.copy('example.txt', 'test_files')
print("After:", glob.glob('test_files/*'))

Before: ['test_files/Untitled Folder', 'test_files/symlink', 'test_files/file', 'test_files/fifo']
After: ['test_files/Untitled Folder', 'test_files/symlink', 'test_files/file', 'test_files/example.txt', 'test_files/fifo']


copy() 方法会像 Unix 命令行工具 cp 那样解释输出文件名称。如果目标是一个目录而不是一个文件，那么将会在这个目录中创建一个与源文件同名的新文件。  
copy2() 类似 copy() ，但是在复制时会包含元数据中的访问和修改时间。  
copymode()可以复制一个文件的权限给新文件， copystat()可以复制文件的其他元数据。

## 处理目录树

In [24]:
import pprint
print('Before:')
pprint.pprint(glob.glob('/tmp/example_dir/*'))
shutil.copytree('../../template_gen', '/tmp/example_dir')
print('After:')
pprint.pprint(glob.glob('/tmp/example_dir/*'))

Before:
[]
After:
['/tmp/example_dir/gen_template.py']


In [23]:
shutil.rmtree('/tmp/example_dir')

In [25]:
print('BEFORE: ', glob.glob('example*'))
shutil.move('example.txt', 'example.out')
print('AFTER : ', glob.glob('example*'))

BEFORE:  ['example_link', 'example.txt', 'example_dir', 'example.txt.copy']
AFTER :  ['example_link', 'example.out', 'example_dir', 'example.txt.copy']


原理类似于 Unix 命令 mv 。如果源文件和目标文件都存在，源文件将会被重命名。否则源文件被复制到目的地然后被删除。

## 查找文件

In [36]:
print(shutil.which('virtualenv'))
print(shutil.which('tox'))
print(shutil.which('git'))
print(shutil.which('example.txt'))
print(shutil.which('python3'))

/usr/local/bin/virtualenv
None
/usr/bin/git
None
/usr/local/bin/python3


which() 方法会按照一个搜索路径查找，典型的使用场景是在环境变量 PATH 定义的路径中查找可执行程序的位置。

In [41]:
path = os.pathsep.join([
    '.',
    os.path.expanduser('~'),
])

mode = os.F_OK | os.R_OK

filename = shutil.which(
    'jmeter.log',
    mode=mode,
    path=path,  # path 参数默认是 os.environ('PATH')，但是可以是任何由 os.pathsep 分隔的字符串
)
print(path)
print(filename)


.:/Users/hejl
/Users/hejl/jmeter.log


## 压缩

In [43]:
for format, des in shutil.get_archive_formats():
    print(f'{format:>5}  {des}')

bztar  bzip2'ed tar-file
gztar  gzip'ed tar-file
  tar  uncompressed tar file
xztar  xz'ed tar-file
  zip  ZIP file


In [48]:
for format, ends, des in shutil.get_unpack_formats():
    print(f'{format :>5} {str(ends): <25} {des}')

bztar ['.tar.bz2', '.tbz2']     bzip2'ed tar-file
gztar ['.tar.gz', '.tgz']       gzip'ed tar-file
  tar ['.tar']                  uncompressed tar file
xztar ['.tar.xz', '.txz']       xz'ed tar-file
  zip ['.zip']                  ZIP file


In [50]:
import logging
import sys
import tarfile

logging.basicConfig(format='%(message)s', stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('pymotw')
print('Creating archive:')
shutil.make_archive('example', 'gztar', root_dir='.', base_dir='test_files', logger=logger)
print('\nArchive contents:')
with tarfile.open('example.tar.gz', 'r') as t:
    for n in t.getnames():
        print(n)

Creating archive:
changing into '.'
Creating tar archive
changing back to '/Users/hejl/local/practise/source_code/standard_library'

Archive contents:
test_files
test_files/Untitled Folder
test_files/Untitled Folder/untitled.txt
test_files/symlink
test_files/file
test_files/example.txt
test_files/fifo


In [53]:
import tempfile
import pathlib
with tempfile.TemporaryDirectory() as d:
    print('Unpacking archive:')
    shutil.unpack_archive('example.tar.gz', extract_dir=d)
    print('\nCreated:')
    prefix_len = len(d)+1
    for extracted in pathlib.Path(d).rglob('*'):
        print(str(extracted))
        print(str(extracted)[prefix_len:])

Unpacking archive:

Created:
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files
test_files
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/Untitled Folder
test_files/Untitled Folder
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/symlink
test_files/symlink
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/file
test_files/file
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/example.txt
test_files/example.txt
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/fifo
test_files/fifo
/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpb_vo4nv2/test_files/Untitled Folder/untitled.txt
test_files/Untitled Folder/untitled.txt


## 文件系统空间

In [56]:
total_b, used_b, free_b = shutil.disk_usage('.')
gib = 2**30 # GiB == gibibyte
gb = 10**9 # GB == gigabyte

print(f"Total: {total_b/gb :6.2f} GB {total_b/gib :6.2f} GiB")
print(f"Used: {used_b/gb :6.2f} GB {used_b/gib :6.2f} GiB")
print(f"Free: {free_b/gb :6.2f} GB {free_b/gib :6.2f} GiB")

Total: 250.69 GB 233.47 GiB
Used:  50.96 GB  47.46 GiB
Free: 195.81 GB 182.36 GiB
